# Example: N-Ary Recombining Lattice Models
In this example, we demonstrate how to implement an N-ary recombining lattice model. An N-ary lattice allows each node to branch into N possible states at each time step, and the lattice recombines at certain points to reduce complexity.

> __Learning Objectives__
>
> In this example, students will learn to: 
> * **N-ary Lattice Construction**: Build multi-branch lattice models from historical stock data using more than two possible price movements per time step. We estimate growth factors and probabilities for each branch to create realistic price trees.
> * **Recombining Tree Implementation**: Construct lattice models that recombine at specific levels to manage computational complexity while maintaining accuracy. We populate the tree with prices and track connectivity between nodes across different time periods.
> * **Price Distribution Analysis**: Analyze possible future price outcomes at specific time horizons using the N-ary lattice structure. We compute probabilities for each possible price level and visualize the distribution of potential returns.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

>__Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [2]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
    Updating registry at `C:\Users\Owner\.julia\registries\General.toml`
   Resolving package versions...
   Installed JpegTurbo ──────────────────────── v0.1.6
   Installed ImageIO ────────────────────────── v0.6.9
   Installed Libmount_jll ───────────────────── v2.41.2+0
   Installed MutableArithmetics ─────────────── v1.6.5
   Installed AxisArrays ─────────────────────── v0.4.8
   Installed ImageSegmentation ──────────────── v1.9.0
   Installed TiledIteration ─────────────────── v0.5.0
   Installed TiffImages ─────────────────────── v0.11.6
   Installed PNGFiles ───────────────────────── v0.4.4
   Installed SciMLPublic ────────────────────── v1.0.0
   Installed ShowCases ──────────────────────── v0.1.0
   Installed HistogramThresholding ──────────── v0.3.1
   Installed FFTW ───────────────────────────── v1.10.0
   Installed ContextVariablesX ──────────────── v0.1.3
   Installed Accessors ───────

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

### Data
We gathered daily open-high-low-close (OHLC) data for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2014` until `12-31-2024`, along with data for a few exchange-traded funds and volatility products during that time period. 

Let's load the `original_dataset::DataFrame` by calling [the `MyTrainingMarketDataSet()` function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/data/#VLQuantitativeFinancePackage.MyTrainingMarketDataSet) and remove firms that do not have the maximum number of trading days. The cleaned dataset $\mathcal{D}$ will be stored in the `dataset` variable.

In [3]:
original_dataset = MyTrainingMarketDataSet() |> x-> x["dataset"];

Not all tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquisition or delisting events. Let's collect only those tickers with the maximum number of trading days.

First, let's compute the number of records for a firm that we know has the maximum value, e.g., `AAPL`, and save that value in the `maximum_number_trading_days::Int64` variable:

In [8]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow # nrow? (check out: DataFrames.jl)

2767

Now, let's iterate through our data and collect only tickers with `maximum_number_trading_days` records. We'll save that data in the `dataset::Dict{String,DataFrame}` variable:

In [10]:
dataset = let

    # initialize -
    dataset = Dict{String, DataFrame}();

    # iterate through the dictionary; we can't guarantee a particular order
    for (ticker, data) ∈ original_dataset  # we get each (K, V) pair!
        if (nrow(data) == maximum_number_trading_days) # what is this doing?
            dataset[ticker] = data;
        end
    end
    dataset; # return
end;

Finally, let's get a list of the firms in our cleaned dataset and sort them alphabetically. We store the sorted firm ticker symbols in the `list_of_tickers::Array{String,1}` variable.

In [12]:
list_of_tickers = keys(dataset) |> collect |> sort # list of firm "ticker" symbols in alphabetical order

424-element Vector{String}:
 "A"
 "AAL"
 "AAP"
 "AAPL"
 "ABBV"
 "ABT"
 "ACN"
 "ADBE"
 "ADI"
 "ADM"
 "ADP"
 "ADSK"
 "AEE"
 ⋮
 "WST"
 "WU"
 "WY"
 "WYNN"
 "XEL"
 "XOM"
 "XRAY"
 "XYL"
 "YUM"
 "ZBRA"
 "ZION"
 "ZTS"

### Constants
Finally, let's set some constants we'll use later in this notebook. The comments describe the constants, their units, and permissible values.

In [14]:
TSIM = 8; # number of trading days to simulate
Δt = (1.0/252); # step size: 1 trading day in units of years
r̄ = 0.05; # risk-free rate (annualized)
n = 3; # number of branches in the N-ary lattice

#### Helper Functions
We define a few helper functions that we'll use later in this notebook.
* The `nodes_at_level(...)` function computes the number of nodes at a given level in an N-ary recombining lattice.
* The `level_offset(...)` function computes the offset to the first node at a given level in an N-ary recombining lattice.

In [16]:
nodes_at_level(i::Integer, n::Integer) = binomial(i + n - 1, i);
level_offset(i::Integer, n::Integer) = i == 0 ? 0 : binomial(i + n - 1, i - 1); # start of level i

___

## Task 1: Compute lattice parameters and future prices from historical data
In this task, we estimate the parameters needed to construct an N-ary recombining lattice model from historical data.

First, we specify the firm we want to analyze. Let's store this in the `selected_firm_ticker::String` variable, get the index of that firm in our `list_of_tickers::Array{String,1}` variable, and extract the firm's historical data from our `dataset::Dict{String,DataFrame}` variable. We'll store the firm's historical data in the `selected_firm_data::DataFrame` variable.

In [19]:
# selected_firm_ticker = rand(list_of_tickers);
selected_firm_ticker = "AAPL"
selected_firm_index = findfirst(x-> x == selected_firm_ticker, list_of_tickers);
selected_firm_data = dataset[selected_firm_ticker];

Next, we specify the `start_index` as the trading day index in the dataset, which will serve as the tree's starting point or `L = 0`. Finally, we set the variable `Sₒ`, which corresponds to the initial price per share at the root of the tree; we use the [volume-weighted average price (VWAP)](https://en.wikipedia.org/wiki/Volume-weighted_average_price) as the initial condition:

In [21]:
#start_index = rand(1:(maximum_number_trading_days - TSIM - 1))
start_index = 1465; # start index for the trading days
stop_index = start_index + TSIM
println("Visualize Firm-$(selected_firm_index) between trading days ($(start_index) -> $(stop_index))")

Visualize Firm-4 between trading days (1465 -> 1473)


We use daily data; thus, the natural time frame between $S_{j-1}$ and $S_{j}$ is a single day. However, it is more convenient to use an annualized value for the $\mu$ parameter; thus, we let $\Delta{t} = 1/252$, i.e., the fraction of a year that corresponds to a single trading day.

The [`log_growth_matrix(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/equity/#VLQuantitativeFinancePackage.log_growth_matrix) takes the cleaned dataset $\mathcal{D}$, which contains $T$ days of data for each firm, a list of firms $\mathcal{L}$, and computes the growth rate values for each firm as a function of time. 

The data is returned as a $(T - 1)\times\dim\mathcal{L}$ array (time on the rows, firm $i$ on the columns). We store the data in the `log_growth_array::Array{Float64,1}` variable:

In [23]:
log_growth_array = log_growth_matrix(dataset, selected_firm_ticker); # array holding growth rate time series

In [24]:
UnicodePlots.histogram(log_growth_array, nbins=21, closed=:left)

                  ┌                                        ┐ 
   [-22.0, -20.0) ┤▎ 3                                       
   [-20.0, -18.0) ┤  0                                       
   [-18.0, -16.0) ┤▏ 1                                       
   [-16.0, -14.0) ┤▎ 3                                       
   [-14.0, -12.0) ┤▌ 11                                      
   [-12.0, -10.0) ┤▋ 13                                      
   [-10.0,  -8.0) ┤█▍ 30                                     
   [ -8.0,  -6.0) ┤███▍ 73                                   
   [ -6.0,  -4.0) ┤███████▎ 159                              
   [ -4.0,  -2.0) ┤█████████████▋ 300                        
   [ -2.0,   0.0) ┤█████████████████████████████▍ 649        
   [  0.0,   2.0) ┤███████████████████████████████████  773  
   [  2.0,   4.0) ┤███████████████████▌ 431                  
   [  4.0,   6.0) ┤████████▊ 194                             
   [  6.0,   8.0) ┤███▏ 67                                   
   [  8.

We've developed the [`build_nary_lattice_from_growth_rate(...)` function](src/Split.jl) to estimate the parameters needed to construct an N-ary recombining lattice model from historical data. The function takes the following arguments:
* `log_growth_array::Array{Float64,2}`: The array of log growth rates for the firms in the dataset.
* `n::Int64`: The number of branches at each node in the N-ary lattice, e.g., `n=2` for a binary lattice, `n=3` for a ternary lattice, etc.
* `dt::Float64`: The time step between levels in the lattice, expressed in years.
* `method::Symbol`: The method used to estimate the lattice parameters; `:quantile` (equal-mass bins) or `:equalwidth` (uniform in growth rate space).

We'll save the output in the `result::NamedTuple` variable.

In [1]:
result = build_nary_lattice_from_growth_rate(log_growth_array; n = n, dt = Δt, method = :equalwidth);

LoadError: UndefVarError: `n` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Let's check out the fields in the `result::NamedTuple` variable.

In [28]:
typeof(result) |> T-> fieldnames(T) # check out the fields of the returned struct

(:edges, :avg_factor, :freq, :counts, :labels, :method, :dt, :N)

The `avg_factor::Array{Float64,1}` field contains the average growth factors associated with each branch at a node in the lattice (arranged in ascending order).

In [30]:
result.avg_factor

3-element Vector{Float64}:
 0.9613877536401365
 1.0011536809331545
 1.0407305793677872

The `freq::Array{Float64,1}` field contains the frequencies associated with each branch at a node in the lattice. 

In [32]:
result.freq

3-element Vector{Float64}:
 0.03109182935647144
 0.940708604483008
 0.028199566160520606

In [33]:
print_lattice(result)

n-ary lattice (method=equalwidth, Δt=0.003968253968253968, N=2766)
State μ-bin [low, high)               avg factor    freq      count
S1    [-21.211458 , -6.905613)        0.961388      0.031092  86
S2    [-6.905613 , 7.400233)          1.001154      0.940709  2602
S3    [7.400233 , 21.706078]          1.040731      0.0282    78


Now, let's build and populate our N-ary recombining lattice model. We'll store the model in the `my_nary_lattice_model::MyGeneralAdjacencyRecombiningCommodityPriceTree` variable.

In [35]:
my_nary_lattice_model = let

    # initialize -
    model = nothing;
    Sₒ = selected_firm_data[start_index,:volume_weighted_average_price];
    Δ = result.avg_factor |> reverse; # average growth factors (up to down)

    # build an empty model -
    model = build(MyGeneralAdjacencyRecombiningCommodityPriceTree, (
        n = n,
        h = TSIM, # how many days to simulate 
    ));

    # populate the data in the model -
    model = populate(model, Sₒ, Δ);

    # print -
    println("Starting price: $(Sₒ) USD for firm $(selected_firm_ticker)");

    model; # return
end;

Starting price: 62.0886 USD for firm AAPL


What's in the `my_nary_lattice_model::MyGeneralAdjacencyRecombiningCommodityPriceTree` instance?

In [37]:
typeof(my_nary_lattice_model) |> T-> fieldnames(T) # check out the fields of the returned struct

(:data, :connectivity, :h, :n)

Let's look at the `connectivity::Dict{Int64,Array{Int64,1}}` field, which contains the adjacency list for the lattice.

In [39]:
my_nary_lattice_model.connectivity

Dict{Int64, Vector{Int64}} with 165 entries:
  5   => [11, 12, 15]
  56  => [84, 85, 92]
  35  => [56, 57, 63]
  55  => [81, 82, 83]
  110 => [150, 151, 155]
  114 => [155, 156, 159]
  123 => [168, 169, 178]
  60  => [88, 89, 96]
  30  => [47, 48, 51]
  32  => [50, 51, 53]
  6   => [12, 13, 16]
  67  => [96, 97, 103]
  45  => [67, 68, 73]
  117 => [159, 160, 162]
  136 => [182, 183, 191]
  145 => [193, 194, 200]
  73  => [103, 104, 109]
  164 => [217, 218, 219]
  115 => [156, 157, 160]
  153 => [202, 203, 208]
  112 => [152, 153, 157]
  64  => [93, 94, 100]
  151 => [200, 201, 206]
  90  => [126, 127, 135]
  139 => [186, 187, 194]
  ⋮   => ⋮

The `data::Dict{Int64, NamedTuple}` field contains a NamedTuple for each node in the lattice, storing relevant information such as the node's price, time, and other attributes.
* The `price::Float64` field contains the price at each node in the lattice.
* The `path::Array{Int64,1}` field contains the path taken to reach each node in the lattice, represented as an array of integers.

Let's take a look at the data.

In [41]:
my_nary_lattice_model.data

Dict{Int64, NamedTuple} with 165 entries:
  5   => (price = 64.6921, path = [1, 1, 0])
  56  => (price = 78.8936, path = [6, 0, 0])
  35  => (price = 75.806, path = [5, 0, 0])
  55  => (price = 50.9923, path = [0, 0, 5])
  110 => (price = 59.789, path = [3, 0, 4])
  114 => (price = 55.2308, path = [2, 0, 5])
  123 => (price = 76.0686, path = [5, 3, 0])
  60  => (price = 67.5603, path = [2, 4, 0])
  30  => (price = 59.7927, path = [1, 1, 2])
  32  => (price = 57.4177, path = [1, 0, 3])
  6   => (price = 62.2319, path = [0, 2, 0])
  67  => (price = 62.4097, path = [1, 4, 1])
  45  => (price = 59.9672, path = [0, 4, 1])
  117 => (price = 51.0201, path = [1, 0, 6])
  136 => (price = 60.1749, path = [0, 7, 1])
  145 => (price = 64.798, path = [4, 1, 3])
  73  => (price = 57.6517, path = [0, 4, 2])
  164 => (price = 45.3107, path = [0, 0, 8])
  115 => (price = 53.1305, path = [1, 1, 5])
  153 => (price = 55.392, path = [1, 3, 4])
  112 => (price = 55.3281, path = [1, 2, 4])
  64  => (price =

## Task 2: Visualize the price distribution at a given level in the lattice
In this task, we visualize the price distribution at a specified level in the N-ary recombining lattice. We'll also compute the probability of the price at each node in the final level of the lattice.

Let's start by specifying the tree level we want to analyze and identifying the nodes at that level. We'll store the nodes at the specified level in the `nodes_at_tree_level::Array{Int64,1}` variable.

In [43]:
nodes_at_tree_level = let

    # initialize -
    l = TSIM; # level
    start = level_offset(l, n);
    stop = start + nodes_at_level(l, n) - 1;

    # compute the nodes indices at level l -
    nodes = range(start, stop=stop, step=1) |> collect;
end

45-element Vector{Int64}:
 120
 121
 122
 123
 124
 125
 126
 127
 128
 129
 130
 131
 132
   ⋮
 153
 154
 155
 156
 157
 158
 159
 160
 161
 162
 163
 164

Next, we compute the price and probability data for each node at the specified level. We extract the price from each node and calculate the probability of reaching that node using the multinomial distribution.

In [45]:
data_array_at_level = let
   
    # initialize -
    number_of_nodes = length(nodes_at_tree_level);
    results_array = Array{Float64, 2}(undef, number_of_nodes, 2);
    Δ = result.avg_factor |> reverse; # average growth factors (up to down)
    p = result.freq |> reverse; # real-world probabilities (up to down)
    d = Multinomial(TSIM, p)

    for i ∈ 1:number_of_nodes
        
        # get node index -
        j = nodes_at_tree_level[i];
        nodemodel = my_nary_lattice_model.data[j];

        # get the price from the node model -
        price = nodemodel.price;

        # ok: let's compute the probability of reaching this node -
        path = nodemodel.path; # path to reach this node
       
        # capture the results -
        results_array[i, 1] = price;
        results_array[i, 2] = pdf(d, path);
    end

    tmp = results_array[:,1];
    sorted_results_array = sortperm(tmp) |> I-> results_array[I, :]; # sort by price
end;

__Check:__ The sum of the probabilities at each level should equal `1.0`. Let's verify this for our specified level:

In [47]:
@assert data_array_at_level[:,2] |> sum ≈ 1.0 # should be 1.0

Let's visualize the possible price distribution at the specified level:

In [49]:
UnicodePlots.histogram(data_array_at_level[:,1], nbins=9, closed=:left)

                ┌                                        ┐ 
   [45.0, 50.0) ┤████████████████▌ 4                       
   [50.0, 55.0) ┤████████████████████▌ 5                   
   [55.0, 60.0) ┤█████████████████████████████████████  9  
   [60.0, 65.0) ┤█████████████████████████████████████  9  
   [65.0, 70.0) ┤████████████████████████▋ 6               
   [70.0, 75.0) ┤████████████████████████▋ 6               
   [75.0, 80.0) ┤████████████████▌ 4                       
   [80.0, 85.0) ┤████▎ 1                                   
   [85.0, 90.0) ┤████▎ 1                                   
                └                                        ┘ 
                                 Frequency                 

Now, let's compute the (multistep) growth rate distributions at the specified level in the lattice. We'll store the data in the `growth_rate_array_at_level::Array{Float64,1}` variable.

In [51]:
growth_rate_array_at_level = let

    # initialize -
    Sₒ = selected_firm_data[start_index,:volume_weighted_average_price];
    number_of_nodes = length(nodes_at_tree_level);
    results_array = Array{Float64, 1}(undef, number_of_nodes);

    for i ∈ 1:number_of_nodes
        
        # get node index -
        j = nodes_at_tree_level[i];
        nodemodel = my_nary_lattice_model.data[j];

        # get the price from the node model -
        price = nodemodel.price;

        # compute the growth rate from the starting price -
        growth_rate = (1/Δt)*log(price/Sₒ);
       
        # capture the results -
        results_array[i] = growth_rate;
    end

    results_array; # return
end;

Let's visualize the growth rate distribution at the specified level:

In [53]:
UnicodePlots.histogram(growth_rate_array_at_level, nbins=21, closed=:left)

                  ┌                                        ┐ 
   [-80.0, -70.0) ┤███████▍ 1                                
   [-70.0, -60.0) ┤███████▍ 1                                
   [-60.0, -50.0) ┤██████████████▊ 2                         
   [-50.0, -40.0) ┤██████████████▊ 2                         
   [-40.0, -30.0) ┤██████████████████████▎ 3                 
   [-30.0, -20.0) ┤██████████████████████▎ 3                 
   [-20.0, -10.0) ┤█████████████████████████████▋ 4          
   [-10.0,   0.0) ┤█████████████████████████████▋ 4          
   [  0.0,  10.0) ┤█████████████████████████████████████  5  
   [ 10.0,  20.0) ┤█████████████████████████████▋ 4          
   [ 20.0,  30.0) ┤█████████████████████████████▋ 4          
   [ 30.0,  40.0) ┤██████████████████████▎ 3                 
   [ 40.0,  50.0) ┤██████████████████████▎ 3                 
   [ 50.0,  60.0) ┤██████████████▊ 2                         
   [ 60.0,  70.0) ┤██████████████▊ 2                         
   [ 70.

___

## Summary
In this notebook, we demonstrated how to construct and analyze N-ary recombining lattice models for stock price evolution using historical data.

> __Key takeaways__
>
> * **N-ary Lattice Construction**: We built multi-branch lattice models that capture more nuanced price movements than traditional binary trees, using three possible states (up, neutral, down) at each node.
> * **Historical Parameter Estimation**: We extracted growth factors and probabilities directly from S&P 500 log returns using equal-width binning to create realistic transition probabilities that reflect actual market behavior.
> * **Recombining Tree Efficiency**: We implemented lattice models that recombine at each level to maintain computational efficiency while preserving essential price dynamics and probability distributions.

Understanding how to build N-ary lattice models provides a more flexible framework for modeling complex financial market movements and price distributions.

___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy, is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.